# UNUSED START--------------------

In [1]:
import pandas as pd

In [2]:
temp = pd.read_csv('UTILIZE-attachments-period_202006091448.csv')

/home/sergey/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
temp.head()

,PCODE,BDATE,FDATE,DATECANCEL,JNAME,SHORTNAME,TNAME
0,90000105,2015-12-19 00:00:00,2100-12-19 00:00:00,NaN,ФЛ,"Скидка:Скидка для ветеранов, инвалидов по обсл...",NaN
1,990130901,2019-05-07 00:00:00,2021-05-06 00:00:00,NaN,Инфоном,NaN,ОМС предоплата
2,90001461,2018-09-15 00:00:00,2020-08-14 00:00:00,2020-08-14 00:00:00,ФЛ,ППП:Доктор рядом - ведение беременности. I-III...,ФЛ предоплата
3,90000090,2013-04-05 00:00:00,2100-04-05 00:00:00,NaN,"ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""СТРАХОВОЕ МЕДИЦ...",NaN,NaN
4,90000013,2015-11-20 00:00:00,2100-11-20 00:00:00,NaN,"ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""ВТБ МЕДИЦИНСКОЕ...",NaN,NaN


In [7]:
# temp.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['PCODE']].count().rename(columns={'PCODE':'Patients'}).reset_index().to_excel('attachments-periods.xlsx', index=False)

In [8]:
temp.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['PCODE']].count().rename(columns={'PCODE':'Patients'}).reset_index()

,JNAME,SHORTNAME,TNAME,Patients
0,"АО ""Башмаковский хлеб""",Диет_Основы питания. Базовый_МКДР,Телемедицина,6
1,"АО ""Башмаковский хлеб""",ТЛЧК_Стандарт. Ежегодное обследование для мужч...,Телемедицина,7
2,ИП Аносенкова Оксана Дмитриевна,Телемедицина_МКДР,Телемедицина,1
3,ИП Глава КФХ Штукатуров Сергей Александрович,Телемедицина_МК,Телемедицина,1
4,ИП Гофман Наталья Викторовна,Диет_Основы питания. Стандарт_МКДР,Телемедицина,1
...,...,...,...,...
1621,ФЛ,Программа на ведение беременности при постанов...,ФЛ предоплата,2
1622,ФЛ,ЧА:Акция у Уролога (3000),ФЛ предоплата,1
1623,Энергогагарант Магнитогорск,ТМ_Семейная_коробка_(2+2)_МКДР,Телемедицина,1
1624,ЮГОРИЯ ЖИЗНЬ,ТМ_Индивид доступ200_деж+узк-2_6 мес_МКДР,Телемедицина,58


In [6]:
# pip install fdb

# UNUSED END -----------

In [7]:
period_start = '01.05.2020'
period_end = '05.05.2020'

In [ ]:
import pandas as pd

In [31]:
import fdb

In [1]:
import sys
sys.path.append("/home/sergey/drclinics/common")

In [2]:
sys.path.append("/home/sergey/drclinics/reports")

In [3]:
from universal_connection import UniversalConnection, DBType

# DISCARDS

In [21]:
sql_discards = f"""

SELECT	clhistnum.PCODE, -- ПАТИЕНТ_ИД
		clhistnum.BDATE, -- НАЧАЛО ПРИКРЕПЛЕНИЯ
		clhistnum.FDATE, -- КОНЕЦ ПРИКРЕПЛЕНИЯ
		clhistnum.DATECANCEL, -- ДОСРОЧНОЕ ОТКРЕПЛЕНИЕ
		jpersons.JNAME, -- ПАРТНЕР
		--TREAT.TREATDATE, -- ДАТА ОКАЗАНИЯ УСЛУГИ
		--TREAT.CREATEDATE,
		--TREAT.TREATCODE, -- APPOINTMENT_ID
		--clhistnum.MAINJID, -- ОСНОВНОЕ ПРИКРЕПЛЕНИЕ
		jplists.shortname, -- ДОГОВОР
		--jpagreement.agnum, -- № ДОГОВОРА
		JPAGRTYPES.TNAME, -- ТИП ДОГОВОРА
		clhistnum.AMOUNTRUB -- ДЕНЬГИ
		--CLHISTNUM.nsp, -- ПОЛИС
		--clients.FIRSTNAME,
		--clients.MIDNAME,
		--clients.LASTNAME,
		--clients.FULLNAME
		--
FROM clhistnum
--LEFT JOIN TREAT ON treat.pcode = clhistnum.PCODE
--
left JOIN jplists ON clhistnum.LSTID = JPLISTS.LID
LEFT JOIN JPAGREEMENT ON JPAGREEMENT.AGRID = CLHISTNUM.AGRID
LEFT JOIN JPAGRTYPES ON JPAGRTYPES.AGRTYPE=JPAGREEMENT.AGRTYPE
LEFT JOIN JPERSONS on JPAGREEMENT.jid = jpersons.JID
--
--LEFT JOIN clients ON clients.PCODE = TREAT.PCODE
--
WHERE 
(clhistnum.FDATE >= '{period_start}' AND clhistnum.FDATE <= '{period_end}') 
OR (clhistnum.DATECANCEL >= '{period_start}' AND clhistnum.DATECANCEL <= '{period_end}' AND clhistnum.FDATE > '{period_end}')


"""

In [23]:
conn = UniversalConnection('../../../../.credentials/infoclinic/PROM.cfg', DBType.FireBird)

df_discards = conn.query(sql_discards)

conn.close()

2020-06-09 16:36:10 connect to firebird database using config file "../../../../.credentials/infoclinic/PROM.cfg"
2020-06-09 16:36:10 connected to database:
                    b:\ifk-fdb\maindb\099.fdb
2020-06-09 16:36:10 query firebird:
                    
				    
				    SELECT	clhistnum.PCODE, -- ПАТИЕНТ_ИД
				    		clhistnum.BDATE, -- НАЧАЛО ПРИКРЕПЛЕНИЯ
				    		clhistnum.FDATE, -- КОНЕЦ ПРИКРЕПЛЕНИЯ
				    		clhistnum.DATECANCEL, -- ДОСРОЧНОЕ ОТКРЕПЛЕНИЕ
				    		jpersons.JNAME, -- ПАРТНЕР
				    		--TREAT.TREATDATE, -- ДАТА ОКАЗАНИЯ УСЛУГИ
				    		--TREAT.CREATEDATE,
				    		--TREAT.TREATCODE, -- APPOINTMENT_ID
				    		--clhistnum.MAINJID, -- ОСНОВНОЕ ПРИКРЕПЛЕНИЕ
				    		jplists.shortname, -- ДОГОВОР
				    		--jpagreement.agnum, -- № ДОГОВОРА
				    		JPAGRTYPES.TNAME, -- ТИП ДОГОВОРА
				    		clhistnum.AMOUNTRUB -- ДЕНЬГИ
				    		--CLHISTNUM.nsp, -- ПОЛИС
				    		--clients.FIRSTNAME,
				    		--clients.MIDNAME,
				    		--clients.LASTNAME,
				    		-

In [24]:
# conn.close()

In [25]:
df_discards

,PCODE,BDATE,FDATE,DATECANCEL,JNAME,SHORTNAME,TNAME,AMOUNTRUB
0,993706276,2019-05-02,2020-05-01,2020-05-01,ФЛ,ППП:Доктор рядом - ведение беременности. II-II...,ФЛ предоплата,48000.0
1,990271579,2018-12-01,2021-04-30,2020-05-01,"АО ""Группа Ренессанс Страхование""",АПП+ПНД+СТ_Ренессанс,СК 1 факт,NaN
2,995509641,2019-05-02,2020-05-01,NaT,"АО ""Русский Стандарт Страхования""",ТМ_Базовый РСС,Телемедицина,425.0
3,990147488,2019-05-06,2020-05-05,NaT,Инфоном,None,ОМС предоплата,NaN
4,990147490,2019-05-06,2020-05-05,NaT,Инфоном,None,ОМС предоплата,NaN
...,...,...,...,...,...,...,...,...
41669,990590027,2020-03-25,2020-05-05,NaT,"Общество с ограниченной ответственностью ""Росг...",АПП_Росгосстрах,СК 1 факт,0.0
41670,990161998,2020-03-25,2020-05-05,NaT,"Общество с ограниченной ответственностью ""Росг...",АПП_Росгосстрах,СК 1 факт,0.0
41671,995773419,2019-06-19,2020-05-03,NaT,"Общество с ограниченной ответственностью ""Росг...",ТМЮЛ1_РГС Телемедицина ЮЛ_МКДР,Телемедицина,50.0
41672,994159202,2019-03-25,2020-05-05,NaT,"Общество с ограниченной ответственностью ""Росг...",ТМЮЛ1_РГС Телемедицина ЮЛ_МКДР,Телемедицина,50.0


In [26]:
df_discards.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['PCODE']].count().reset_index().rename(columns={'PCODE':'DISCARDS'})

,JNAME,SHORTNAME,TNAME,DISCARDS
0,"АО ""Группа Ренессанс Страхование""",АПП+ПНД дети 11-18_Ренес,СК 1 факт,1
1,"АО ""Группа Ренессанс Страхование""",АПП+ПНД+СТ дети 11-18_Ренес,СК 1 факт,1
2,"АО ""Группа Ренессанс Страхование""",АПП+ПНД+СТ+ТМ_Ренесс,СК 1 факт,24
3,"АО ""Группа Ренессанс Страхование""",АПП+ПНД+СТ_Ренессанс,СК 1 факт,39
4,"АО ""Группа Ренессанс Страхование""",АПП+ПНД+СТ_дети_Ренессанс,СК 1 факт,2
...,...,...,...,...
120,"Страховое публичное акционерное общество ""РЕСО...",АПП+ПНД+СТ_РесоГарантия,СК 1 факт,1
121,"Страховое публичное акционерное общество ""РЕСО...",АПП+ПНД_РесоГарантия,СК 1 факт,19
122,Техническая программа,ТМ_Промокод_все врачи,Телемедицина,46
123,Техническая программа,ТМ_Промокод_дежурные,Телемедицина,1


In [47]:
discards = df_discards.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['PCODE']].count().reset_index().rename(columns={'PCODE':'DISCARDS'})

# NEW PATIENTS

In [28]:
sql_joins = f"""

SELECT	clhistnum.PCODE, -- ПАТИЕНТ_ИД
		clhistnum.BDATE, -- НАЧАЛО ПРИКРЕПЛЕНИЯ
		clhistnum.FDATE, -- КОНЕЦ ПРИКРЕПЛЕНИЯ
		clhistnum.DATECANCEL, -- ДОСРОЧНОЕ ОТКРЕПЛЕНИЕ
		jpersons.JNAME, -- ПАРТНЕР
		--TREAT.TREATDATE, -- ДАТА ОКАЗАНИЯ УСЛУГИ
		--TREAT.CREATEDATE,
		--TREAT.TREATCODE, -- APPOINTMENT_ID
		--clhistnum.MAINJID, -- ОСНОВНОЕ ПРИКРЕПЛЕНИЕ
		jplists.shortname, -- ДОГОВОР
		--jpagreement.agnum, -- № ДОГОВОРА
		JPAGRTYPES.TNAME, -- ТИП ДОГОВОРА
		clhistnum.AMOUNTRUB -- ДЕНЬГИ
		--CLHISTNUM.nsp, -- ПОЛИС
		--clients.FIRSTNAME,
		--clients.MIDNAME,
		--clients.LASTNAME,
		--clients.FULLNAME
		--
FROM clhistnum
--LEFT JOIN TREAT ON treat.pcode = clhistnum.PCODE
--
left JOIN jplists ON clhistnum.LSTID = JPLISTS.LID
LEFT JOIN JPAGREEMENT ON JPAGREEMENT.AGRID = CLHISTNUM.AGRID
LEFT JOIN JPAGRTYPES ON JPAGRTYPES.AGRTYPE=JPAGREEMENT.AGRTYPE
LEFT JOIN JPERSONS on JPAGREEMENT.jid = jpersons.JID
--
--LEFT JOIN clients ON clients.PCODE = TREAT.PCODE
--
WHERE 
clhistnum.BDATE >= '{period_start}' AND clhistnum.BDATE <= '{period_end}'

"""

In [29]:
conn = UniversalConnection('../../../../.credentials/infoclinic/PROM.cfg', DBType.FireBird)

df_joins = conn.query(sql_joins)

conn.close()

2020-06-09 16:40:17 connect to firebird database using config file "../../../../.credentials/infoclinic/PROM.cfg"
2020-06-09 16:40:17 connected to database:
                    b:\ifk-fdb\maindb\099.fdb
2020-06-09 16:40:17 query firebird:
                    
				    
				    SELECT	clhistnum.PCODE, -- ПАТИЕНТ_ИД
				    		clhistnum.BDATE, -- НАЧАЛО ПРИКРЕПЛЕНИЯ
				    		clhistnum.FDATE, -- КОНЕЦ ПРИКРЕПЛЕНИЯ
				    		clhistnum.DATECANCEL, -- ДОСРОЧНОЕ ОТКРЕПЛЕНИЕ
				    		jpersons.JNAME, -- ПАРТНЕР
				    		--TREAT.TREATDATE, -- ДАТА ОКАЗАНИЯ УСЛУГИ
				    		--TREAT.CREATEDATE,
				    		--TREAT.TREATCODE, -- APPOINTMENT_ID
				    		--clhistnum.MAINJID, -- ОСНОВНОЕ ПРИКРЕПЛЕНИЕ
				    		jplists.shortname, -- ДОГОВОР
				    		--jpagreement.agnum, -- № ДОГОВОРА
				    		JPAGRTYPES.TNAME, -- ТИП ДОГОВОРА
				    		clhistnum.AMOUNTRUB -- ДЕНЬГИ
				    		--CLHISTNUM.nsp, -- ПОЛИС
				    		--clients.FIRSTNAME,
				    		--clients.MIDNAME,
				    		--clients.LASTNAME,
				    		-

In [30]:
df_joins

,PCODE,BDATE,FDATE,DATECANCEL,JNAME,SHORTNAME,TNAME,AMOUNTRUB
0,990177610,2020-05-01,2021-04-30,NaT,Инфоном,None,ОМС предоплата,NaN
1,990369431,2020-05-01,2020-08-31,2020-05-06,"Открытое акционерное общество ""АльфаСтрахование""",АПП_Альфастрах,СК 1 факт,0.0
2,990375026,2020-05-01,2020-08-31,NaT,"Открытое акционерное общество ""АльфаСтрахование""",АПП_Альфастрах,СК 1 факт,0.0
3,990369476,2020-05-01,2020-08-31,NaT,"Открытое акционерное общество ""АльфаСтрахование""",АПП_Альфастрах,СК 1 факт,0.0
4,990185159,2020-05-02,2021-05-01,NaT,Инфоном,None,ОМС предоплата,NaN
...,...,...,...,...,...,...,...,...
59310,997992419,2020-05-05,2020-06-04,NaT,Содружество,ТМ_Здоровье с Деньги Сразу до 2000_МКДР,Телемедицина,45.0
59311,998645543,2020-05-05,2020-06-04,NaT,Содружество,ТМ_Здоровье с Деньги Сразу до 2000_МКДР,Телемедицина,45.0
59312,997357483,2020-05-05,2020-06-04,NaT,Содружество,ТМ_Здоровье с Деньги Сразу до 2000_МКДР,Телемедицина,45.0
59313,998645574,2020-05-05,2020-06-04,NaT,Содружество,ТМ_Здоровье с Деньги Сразу до 2000_МКДР,Телемедицина,45.0


In [31]:
df_joins.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['PCODE']].count().reset_index().rename(columns={'PCODE':'JOINS'})

,JNAME,SHORTNAME,TNAME,JOINS
0,"АО ""Группа Ренессанс Страхование""",АПП+ПНД+СТ дети 11-18_Ренес,СК 1 факт,2
1,"АО ""Группа Ренессанс Страхование""",АПП+ПНД+СТ+ТМ_Ренесс,СК 1 факт,1
2,"АО ""Группа Ренессанс Страхование""",АПП+ПНД+ТМ_Ренесс,СК 1 факт,400
3,"АО ""Группа Ренессанс Страхование""",АПП+ТМ_Ренесс,СК 1 факт,1826
4,"АО ""Группа Ренессанс Страхование""",Второе медицинское мнение ТМ_МКДР,Телемедицина,3
...,...,...,...,...
102,"Страховое публичное акционерное общество ""РЕСО...",АПП+ПНД_РесоГарантия,СК 1 факт,1251
103,"Страховое публичное акционерное общество ""РЕСО...",АПП+СТ_РесоГарантия,СК 1 факт,1
104,Техническая программа,ТМ_Промокод_все врачи,Телемедицина,230
105,ЮГОРИЯ ЖИЗНЬ,ТМ_Индивид доступ200_деж+узк-2_6 мес_МКДР,Телемедицина,42


In [39]:
joins = df_joins.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['PCODE']].count().reset_index().rename(columns={'PCODE':'JOINS'})

# CURRENT PATIENTS

In [33]:
sql_current = f"""
SELECT	clhistnum.PCODE, -- ПАТИЕНТ_ИД
		clhistnum.BDATE, -- НАЧАЛО ПРИКРЕПЛЕНИЯ
		clhistnum.FDATE, -- КОНЕЦ ПРИКРЕПЛЕНИЯ
		clhistnum.DATECANCEL, -- ДОСРОЧНОЕ ОТКРЕПЛЕНИЕ
		jpersons.JNAME, -- ПАРТНЕР
		--TREAT.TREATDATE, -- ДАТА ОКАЗАНИЯ УСЛУГИ
		--TREAT.CREATEDATE,
		--TREAT.TREATCODE, -- APPOINTMENT_ID
		--clhistnum.MAINJID, -- ОСНОВНОЕ ПРИКРЕПЛЕНИЕ
		jplists.shortname, -- ДОГОВОР
		--jpagreement.agnum, -- № ДОГОВОРА
		JPAGRTYPES.TNAME, -- ТИП ДОГОВОРА
		clhistnum.AMOUNTRUB -- ДЕНЬГИ
		--CLHISTNUM.nsp, -- ПОЛИС
		--clients.FIRSTNAME,
		--clients.MIDNAME,
		--clients.LASTNAME,
		--clients.FULLNAME
		--
FROM clhistnum
--LEFT JOIN TREAT ON treat.pcode = clhistnum.PCODE
--
left JOIN jplists ON clhistnum.LSTID = JPLISTS.LID
LEFT JOIN JPAGREEMENT ON JPAGREEMENT.AGRID = CLHISTNUM.AGRID
LEFT JOIN JPAGRTYPES ON JPAGRTYPES.AGRTYPE=JPAGREEMENT.AGRTYPE
LEFT JOIN JPERSONS on JPAGREEMENT.jid = jpersons.JID
--
--LEFT JOIN clients ON clients.PCODE = TREAT.PCODE
--
WHERE (
clhistnum.BDATE <= '{period_end}' --period end
AND clhistnum.FDATE > '{period_start}' --period start
AND (clhistnum.DATECANCEL IS NULL OR clhistnum.FDATE <= clhistnum.DATECANCEL))


"""

In [34]:
conn = UniversalConnection('../../../../.credentials/infoclinic/PROM.cfg', DBType.FireBird)

df_current = conn.query(sql_current)

conn.close()

2020-06-09 16:44:55 connect to firebird database using config file "../../../../.credentials/infoclinic/PROM.cfg"
2020-06-09 16:44:56 connected to database:
                    b:\ifk-fdb\maindb\099.fdb
2020-06-09 16:44:56 query firebird:
                    
				    SELECT	clhistnum.PCODE, -- ПАТИЕНТ_ИД
				    		clhistnum.BDATE, -- НАЧАЛО ПРИКРЕПЛЕНИЯ
				    		clhistnum.FDATE, -- КОНЕЦ ПРИКРЕПЛЕНИЯ
				    		clhistnum.DATECANCEL, -- ДОСРОЧНОЕ ОТКРЕПЛЕНИЕ
				    		jpersons.JNAME, -- ПАРТНЕР
				    		--TREAT.TREATDATE, -- ДАТА ОКАЗАНИЯ УСЛУГИ
				    		--TREAT.CREATEDATE,
				    		--TREAT.TREATCODE, -- APPOINTMENT_ID
				    		--clhistnum.MAINJID, -- ОСНОВНОЕ ПРИКРЕПЛЕНИЕ
				    		jplists.shortname, -- ДОГОВОР
				    		--jpagreement.agnum, -- № ДОГОВОРА
				    		JPAGRTYPES.TNAME, -- ТИП ДОГОВОРА
				    		clhistnum.AMOUNTRUB -- ДЕНЬГИ
				    		--CLHISTNUM.nsp, -- ПОЛИС
				    		--clients.FIRSTNAME,
				    		--clients.MIDNAME,
				    		--clients.LASTNAME,
				    		--clients.

In [35]:
df_current

,PCODE,BDATE,FDATE,DATECANCEL,JNAME,SHORTNAME,TNAME,AMOUNTRUB
0,90000105,2015-12-19,2100-12-19 00:00:00,NaT,ФЛ,"Скидка:Скидка для ветеранов, инвалидов по обсл...",None,NaN
1,990130901,2019-05-07,2021-05-06 00:00:00,NaT,Инфоном,None,ОМС предоплата,NaN
2,90001461,2018-09-15,2020-08-14 00:00:00,2020-08-14,ФЛ,ППП:Доктор рядом - ведение беременности. I-III...,ФЛ предоплата,63000.0
3,90000090,2013-04-05,2100-04-05 00:00:00,NaT,"ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""СТРАХОВОЕ МЕДИЦ...",None,None,NaN
4,90000013,2015-11-20,2100-11-20 00:00:00,NaT,"ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""ВТБ МЕДИЦИНСКОЕ...",None,None,NaN
...,...,...,...,...,...,...,...,...
5955246,997992419,2020-05-05,2020-06-04 00:00:00,NaT,Содружество,ТМ_Здоровье с Деньги Сразу до 2000_МКДР,Телемедицина,45.0
5955247,998645543,2020-05-05,2020-06-04 00:00:00,NaT,Содружество,ТМ_Здоровье с Деньги Сразу до 2000_МКДР,Телемедицина,45.0
5955248,997357483,2020-05-05,2020-06-04 00:00:00,NaT,Содружество,ТМ_Здоровье с Деньги Сразу до 2000_МКДР,Телемедицина,45.0
5955249,998645574,2020-05-05,2020-06-04 00:00:00,NaT,Содружество,ТМ_Здоровье с Деньги Сразу до 2000_МКДР,Телемедицина,45.0


In [36]:
df_current.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['PCODE']].count().reset_index().rename(columns={'PCODE':'CURRENT'})

,JNAME,SHORTNAME,TNAME,CURRENT
0,"АО ""Башмаковский хлеб""",Диет_Основы питания. Базовый_МКДР,Телемедицина,6
1,"АО ""Башмаковский хлеб""",ТЛЧК_Стандарт. Ежегодное обследование для мужч...,Телемедицина,7
2,ИП Аносенкова Оксана Дмитриевна,Телемедицина_МКДР,Телемедицина,1
3,ИП Глава КФХ Штукатуров Сергей Александрович,Телемедицина_МК,Телемедицина,1
4,ИП Гофман Наталья Викторовна,Диет_Основы питания. Стандарт_МКДР,Телемедицина,1
...,...,...,...,...
1621,ФЛ,Программа на ведение беременности при постанов...,ФЛ предоплата,2
1622,ФЛ,ЧА:Акция у Уролога (3000),ФЛ предоплата,1
1623,Энергогагарант Магнитогорск,ТМ_Семейная_коробка_(2+2)_МКДР,Телемедицина,1
1624,ЮГОРИЯ ЖИЗНЬ,ТМ_Индивид доступ200_деж+узк-2_6 мес_МКДР,Телемедицина,58


In [37]:
current = df_current.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['PCODE']].count().reset_index().rename(columns={'PCODE':'CURRENT_TEMP'})

In [49]:
current.head()

,JNAME,SHORTNAME,TNAME,CURRENT_TEMP
0,"АО ""Башмаковский хлеб""",Диет_Основы питания. Базовый_МКДР,Телемедицина,6
1,"АО ""Башмаковский хлеб""",ТЛЧК_Стандарт. Ежегодное обследование для мужч...,Телемедицина,7
2,ИП Аносенкова Оксана Дмитриевна,Телемедицина_МКДР,Телемедицина,1
3,ИП Глава КФХ Штукатуров Сергей Александрович,Телемедицина_МК,Телемедицина,1
4,ИП Гофман Наталья Викторовна,Диет_Основы питания. Стандарт_МКДР,Телемедицина,1


In [43]:
# joins[joins['JNAME'] == 'АО "Группа Ренессанс Страхование"']

In [44]:
# current[current['JNAME'] == 'АО "Группа Ренессанс Страхование"']

In [58]:
merge_temp = current.merge(joins, how='left')
merge_temp.JOINS = merge_temp.JOINS.fillna(0).astype(int)

In [59]:
merged = merge_temp.merge(discards, how='left')
merged.DISCARDS = merged.DISCARDS.fillna(0).astype(int)

In [64]:
merged['ON_START'] = merged['CURRENT_TEMP'] + merged['DISCARDS']
merged['ON_END'] = merged['CURRENT_TEMP'] - merged['DISCARDS'] + merged['JOINS']

In [68]:
merged.tail(10)

,JNAME,SHORTNAME,TNAME,CURRENT_TEMP,JOINS,DISCARDS,ON_START,ON_END
1616,ФЛ,"ППП:Программа медобслуживания для взрослых ""Ба...",ФЛ предоплата,3,0,0,3,3
1617,ФЛ,"ППП:Программа медобслуживания для взрослых ""Ба...",ФЛ предоплата,2,0,0,2,2
1618,ФЛ,"ППП:Программа медобслуживания для взрослых ""Ст...",ФЛ предоплата,40,0,0,40,40
1619,ФЛ,ППП:Программа медобслуживания для взрослых Ста...,ФЛ предоплата,6,0,0,6,6
1620,ФЛ,ППП:Программа медобслуживания для взрослых Ста...,ФЛ предоплата,2,0,0,2,2
1621,ФЛ,Программа на ведение беременности при постанов...,ФЛ предоплата,2,0,0,2,2
1622,ФЛ,ЧА:Акция у Уролога (3000),ФЛ предоплата,1,0,0,1,1
1623,Энергогагарант Магнитогорск,ТМ_Семейная_коробка_(2+2)_МКДР,Телемедицина,1,0,0,1,1
1624,ЮГОРИЯ ЖИЗНЬ,ТМ_Индивид доступ200_деж+узк-2_6 мес_МКДР,Телемедицина,58,42,0,58,100
1625,ЮГОРИЯ ЖИЗНЬ,ТМ_Индивид доступ285_деж+узк_6 мес_МКДР,Телемедицина,1,1,0,1,2


In [67]:
merged.to_excel('merged.xlsx', index=False)

# CONSULTATIONS

In [69]:
sql_cons = f"""
SELECT 	TREAT.TREATCODE,
		TREAT.TREATDATE,
		--clhistnum.MAINJID,
		--clhistnum.PARENTHISTID,
		jpersons.JNAME,
		clhistnum.PCODE, -- ПАТИЕНТ_ИД
		--clhistnum.BDATE, -- НАЧАЛО ПРИКРЕПЛЕНИЯ
		--clhistnum.FDATE, -- КОНЕЦ ПРИКРЕПЛЕНИЯ
		--clhistnum.DATECANCEL, -- ДОСРОЧНОЕ ОТКРЕПЛЕНИЕ
		jplists.shortname, -- ДОГОВОР
		jpagreement.agnum, -- № ДОГОВОРА
		JPAGRTYPES.TNAME--, -- ТИП ДОГОВОРА
		--clhistnum.AMOUNTRUB -- ДЕНЬГИ
FROM TREAT
INNER JOIN clhistnum ON TREAT.HISTID = CLHISTNUM.HISTID 
					 AND TREAT.PCODE = CLHISTNUM.PCODE 
--inner JOIN clients ON TREAT.PCODE = clients.PCODE 
--JOIN JPERSONS ON TREAT.CLINICID = jpersons.jid
INNER JOIN JPERSONS ON CLHISTNUM.JID = jpersons.JID 
--
--
--left JOIN CLHISTNUM ON JPAGREEMENT.AGRID = CLHISTNUM.AGRID
inner JOIN jplists ON clhistnum.LSTID = JPLISTS.LID
--inner JOIN JPAGREEMENT ON JPAGREEMENT.jid = jpersons.JID
--
INNER JOIN jpagreement ON CLHISTNUM.AGRID = jpagreement.AGRID	
--
inner JOIN JPAGRTYPES ON JPAGRTYPES.AGRTYPE = JPAGREEMENT.AGRTYPE
--
--WHERE TREAT.TREATCODE = 60595284
WHERE (TREAT.TREATDATE >= '{period_start}'
AND TREAT.TREATDATE <= '{period_end}') --OR (TREAT.TREATDATE IS NULL AND clhistnum.PCODE IS NOT NULL))
--
AND (clhistnum.FDATE > '{period_start}' AND (clhistnum.DATECANCEL IS NULL OR TREAT.TREATDATE <= clhistnum.DATECANCEL))



"""

In [70]:
conn = UniversalConnection('../../../../.credentials/infoclinic/PROM.cfg', DBType.FireBird)

df_cons = conn.query(sql_cons)

conn.close()

2020-06-10 18:59:00 connect to firebird database using config file "../../../../.credentials/infoclinic/PROM.cfg"
2020-06-10 18:59:02 connected to database:
                    b:\ifk-fdb\maindb\099.fdb
2020-06-10 18:59:02 query firebird:
                    
				    SELECT 	TREAT.TREATCODE,
				    		TREAT.TREATDATE,
				    		--clhistnum.MAINJID,
				    		--clhistnum.PARENTHISTID,
				    		jpersons.JNAME,
				    		clhistnum.PCODE, -- ПАТИЕНТ_ИД
				    		--clhistnum.BDATE, -- НАЧАЛО ПРИКРЕПЛЕНИЯ
				    		--clhistnum.FDATE, -- КОНЕЦ ПРИКРЕПЛЕНИЯ
				    		--clhistnum.DATECANCEL, -- ДОСРОЧНОЕ ОТКРЕПЛЕНИЕ
				    		jplists.shortname, -- ДОГОВОР
				    		jpagreement.agnum, -- № ДОГОВОРА
				    		JPAGRTYPES.TNAME--, -- ТИП ДОГОВОРА
				    		--clhistnum.AMOUNTRUB -- ДЕНЬГИ
				    FROM TREAT
				    INNER JOIN clhistnum ON TREAT.HISTID = CLHISTNUM.HISTID 
				    					 AND TREAT.PCODE = CLHISTNUM.PCODE 
				    --inner JOIN clients ON TREAT.PCODE = clients.PCODE 
				    --JOIN JPER

In [72]:
# conn.close()

In [73]:
df_cons

,TREATCODE,TREATDATE,JNAME,PCODE,SHORTNAME,AGNUM,TNAME
0,21176929,2020-05-01,"АО ""Группа Ренессанс Страхование""",996796738,Телемедицина_Ренессанс МКДР,МКДР ДС№1 от 14.06.2019 к Дог. №30-19 от 01.06...,Телемедицина
1,21176939,2020-05-01,"Страховое публичное акционерное общество ""РЕСО...",990342288,АПП+ПНД_РесоГарантия,14/53,СК 1 факт
2,42017309,2020-05-01,"АО ""Группа Ренессанс Страхование""",994877793,АПП_Ренессанс,54/15,СК 1 факт
3,60594885,2020-05-01,"АО ""Группа Ренессанс Страхование""",994320877,АПП+ПНД+ТМ_Ренесс,54/15,СК 1 факт
4,70889950,2020-05-01,"АО ""Группа Ренессанс Страхование""",995186266,АПП+ПНД+ТМ_Ренесс,54/15,СК 1 факт
...,...,...,...,...,...,...,...
2992,997931268,2020-05-04,"АО ""Группа Ренессанс Страхование""",990565780,ТМ ДЛЯ СОТРУДНИКОВ_СТАНДАРТ_МКДР,ДС №1 от 28.11.2019г. к Дог.№69-19 от 28.11.20...,Телемедицина
2993,997936784,2020-05-01,Техническая программа,991154240,ТМ_Промокод_все врачи,1_технический,Телемедицина
2994,997938550,2020-05-02,"Общество с ограниченной ответственностью СК ""А...",995794051,Альянс Жизнь_взрослый_ТМ,МК ДР_Дог.№Л-135219/NV-032 ОТ 29.11.2019_(факт...,Телемедицина
2995,997938555,2020-05-05,Техническая программа,998065823,ТМ_Промокод_все врачи,1_технический,Телемедицина


In [74]:
df_cons.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['TREATCODE']].count().reset_index().rename(columns={'TREATCODE':'CONS_AMOUNT'})

,JNAME,SHORTNAME,TNAME,CONS_AMOUNT
0,"АО ""Группа Ренессанс Страхование""",А АПП (10-50 чел),СК 1 предоплата,4
1,"АО ""Группа Ренессанс Страхование""",АПП+ПНД+СТ_Ренессанс,СК 1 факт,15
2,"АО ""Группа Ренессанс Страхование""",АПП+ПНД+ТМ_Ренесс,СК 1 факт,96
3,"АО ""Группа Ренессанс Страхование""",АПП+ПНД_Ренессанс,СК 1 факт,60
4,"АО ""Группа Ренессанс Страхование""",АПП+ТМ_Ренесс,СК 1 факт,169
...,...,...,...,...
115,Техническая программа,ТМ_Промокод_все врачи,Телемедицина,410
116,Техническая программа,ТМ_Промокод_дежурные,Телемедицина,5
117,ФЛ,ППП:Доктор рядом - ведение беременности. I-II ...,ФЛ предоплата,1
118,ФЛ,ППП:Доктор рядом - ведение беременности. I-III...,ФЛ предоплата,17


In [75]:
cons = df_cons.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['TREATCODE']].count().reset_index().rename(columns={'TREATCODE':'CONS_AMOUNT'})

In [76]:
merged = merged.merge(cons, how='left')
merged.CONS_AMOUNT = merged.CONS_AMOUNT.fillna(0).astype(int)

In [81]:
merged.CONS_AMOUNT.sum()

2997

In [82]:
merged

,JNAME,SHORTNAME,TNAME,CURRENT_TEMP,JOINS,DISCARDS,ON_START,ON_END,CONS_AMOUNT
0,"АО ""Башмаковский хлеб""",Диет_Основы питания. Базовый_МКДР,Телемедицина,6,0,0,6,6,0
1,"АО ""Башмаковский хлеб""",ТЛЧК_Стандарт. Ежегодное обследование для мужч...,Телемедицина,7,0,0,7,7,0
2,ИП Аносенкова Оксана Дмитриевна,Телемедицина_МКДР,Телемедицина,1,0,0,1,1,0
3,ИП Глава КФХ Штукатуров Сергей Александрович,Телемедицина_МК,Телемедицина,1,0,0,1,1,0
4,ИП Гофман Наталья Викторовна,Диет_Основы питания. Стандарт_МКДР,Телемедицина,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...
1621,ФЛ,Программа на ведение беременности при постанов...,ФЛ предоплата,2,0,0,2,2,4
1622,ФЛ,ЧА:Акция у Уролога (3000),ФЛ предоплата,1,0,0,1,1,0
1623,Энергогагарант Магнитогорск,ТМ_Семейная_коробка_(2+2)_МКДР,Телемедицина,1,0,0,1,1,0
1624,ЮГОРИЯ ЖИЗНЬ,ТМ_Индивид доступ200_деж+узк-2_6 мес_МКДР,Телемедицина,58,42,0,58,100,0


In [103]:
len(df_cons.PCODE.unique())

1722

In [100]:
df_cons.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['PCODE']].nunique().reset_index().rename(columns={'PCODE':'UNIQUE_USERS'})

,JNAME,SHORTNAME,TNAME,UNIQUE_USERS
0,"АО ""Группа Ренессанс Страхование""",А АПП (10-50 чел),СК 1 предоплата,2
1,"АО ""Группа Ренессанс Страхование""",АПП+ПНД+СТ_Ренессанс,СК 1 факт,5
2,"АО ""Группа Ренессанс Страхование""",АПП+ПНД+ТМ_Ренесс,СК 1 факт,42
3,"АО ""Группа Ренессанс Страхование""",АПП+ПНД_Ренессанс,СК 1 факт,23
4,"АО ""Группа Ренессанс Страхование""",АПП+ТМ_Ренесс,СК 1 факт,64
...,...,...,...,...
115,Техническая программа,ТМ_Промокод_все врачи,Телемедицина,317
116,Техническая программа,ТМ_Промокод_дежурные,Телемедицина,4
117,ФЛ,ППП:Доктор рядом - ведение беременности. I-II ...,ФЛ предоплата,1
118,ФЛ,ППП:Доктор рядом - ведение беременности. I-III...,ФЛ предоплата,9


In [101]:
unique_users = df_cons.groupby(['JNAME', 'SHORTNAME', 'TNAME'])[['PCODE']].nunique().reset_index().rename(columns={'PCODE':'UNIQUE_USERS'})

In [104]:
unique_users.UNIQUE_USERS.sum()

1738

In [105]:
merged = merged.merge(unique_users, how='left')
merged.UNIQUE_USERS = merged.UNIQUE_USERS.fillna(0).astype(int)

In [109]:
merged = merged.drop(['CURRENT_TEMP'], axis=1)

In [111]:
merged

,JNAME,SHORTNAME,TNAME,JOINS,DISCARDS,ON_START,ON_END,CONS_AMOUNT,UNIQUE_USERS
0,"АО ""Башмаковский хлеб""",Диет_Основы питания. Базовый_МКДР,Телемедицина,0,0,6,6,0,0
1,"АО ""Башмаковский хлеб""",ТЛЧК_Стандарт. Ежегодное обследование для мужч...,Телемедицина,0,0,7,7,0,0
2,ИП Аносенкова Оксана Дмитриевна,Телемедицина_МКДР,Телемедицина,0,0,1,1,0,0
3,ИП Глава КФХ Штукатуров Сергей Александрович,Телемедицина_МК,Телемедицина,0,0,1,1,0,0
4,ИП Гофман Наталья Викторовна,Диет_Основы питания. Стандарт_МКДР,Телемедицина,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...
1621,ФЛ,Программа на ведение беременности при постанов...,ФЛ предоплата,0,0,2,2,4,1
1622,ФЛ,ЧА:Акция у Уролога (3000),ФЛ предоплата,0,0,1,1,0,0
1623,Энергогагарант Магнитогорск,ТМ_Семейная_коробка_(2+2)_МКДР,Телемедицина,0,0,1,1,0,0
1624,ЮГОРИЯ ЖИЗНЬ,ТМ_Индивид доступ200_деж+узк-2_6 мес_МКДР,Телемедицина,42,0,58,100,0,0


In [110]:
# merged.to_excel('UTILIZE.xlsx')

In [116]:
merged['CONS_ON_USER'] = 

In [117]:
merged['UTILIZE'] = (merged['CONS_AMOUNT'] / ((merged['ON_END'] - merged['ON_START']) + merged['ON_START']))

In [118]:
merged

,JNAME,SHORTNAME,TNAME,JOINS,DISCARDS,ON_START,ON_END,CONS_AMOUNT,UNIQUE_USERS,UTILIZE
0,"АО ""Башмаковский хлеб""",Диет_Основы питания. Базовый_МКДР,Телемедицина,0,0,6,6,0,0,0.0
1,"АО ""Башмаковский хлеб""",ТЛЧК_Стандарт. Ежегодное обследование для мужч...,Телемедицина,0,0,7,7,0,0,0.0
2,ИП Аносенкова Оксана Дмитриевна,Телемедицина_МКДР,Телемедицина,0,0,1,1,0,0,0.0
3,ИП Глава КФХ Штукатуров Сергей Александрович,Телемедицина_МК,Телемедицина,0,0,1,1,0,0,0.0
4,ИП Гофман Наталья Викторовна,Диет_Основы питания. Стандарт_МКДР,Телемедицина,0,0,1,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...
1621,ФЛ,Программа на ведение беременности при постанов...,ФЛ предоплата,0,0,2,2,4,1,2.0
1622,ФЛ,ЧА:Акция у Уролога (3000),ФЛ предоплата,0,0,1,1,0,0,0.0
1623,Энергогагарант Магнитогорск,ТМ_Семейная_коробка_(2+2)_МКДР,Телемедицина,0,0,1,1,0,0,0.0
1624,ЮГОРИЯ ЖИЗНЬ,ТМ_Индивид доступ200_деж+узк-2_6 мес_МКДР,Телемедицина,42,0,58,100,0,0,0.0


In [119]:
merged = merged.drop(['UTILIZE'], axis=1)